In [1]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='lnumerow', api_key='mDhzegrhH0qf8rlFbzOo')
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math

In [2]:
datafile = r'''Data\IVE_bidask1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskLow', 'AskHigh', 'AskClose']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

In [3]:
pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2019,1,1,0,0,0)].copy()
pricedata = pricedata.reset_index()
del pricedata['index']

In [4]:
def changeTimeframe(pricedata, timeframe_mins):
    newpricedata = pd.DataFrame()
    
    index = 0
    
    while(index < len(pricedata)):
        newcandle = pricedata.iloc[index].copy()
        starttime = newcandle.DateTime
        
        while(True):
            index = index + 1
            if index == len(pricedata):
                break
            
            candle = pricedata.iloc[index].copy()
            
            if candle.DateTime - starttime >= datetime.timedelta(minutes=timeframe_mins):
                break
                
            newcandle.AskClose = candle.AskClose
            newcandle.AskHigh = max(newcandle.AskHigh, candle.AskHigh)
            newcandle.AskLow = min(newcandle.AskLow, candle.AskLow)
            newcandle.BidClose = candle.BidClose
            newcandle.BidHigh = max(newcandle.BidHigh, candle.BidHigh)
            newcandle.BidLow = min(newcandle.BidLow, candle.BidLow)
        
        newpricedata = newpricedata.append(newcandle)
        
    return newpricedata

In [5]:
class CandlestickChart:
    
    def updateChart(self, placeholder):
        starttime = datetime.datetime.combine(self.startdatewidget.value, datetime.time())
        endtime = datetime.datetime.combine(self.enddatewidget.value, datetime.time()) + datetime.timedelta(days=1)
        timeframe = self.timeframewidget.value
        
        fullpricedata = self.fullpricedata
        chartdata = fullpricedata[fullpricedata['DateTime'] >= starttime]
        chartdata = fullpricedata[fullpricedata['DateTime'] < endtime]
        chartdata = changeTimeframe(chartdata, timeframe).copy()
        
        data = self.chart.data[0]
        data.x = [ii for ii in range(len(chartdata['BidLow']))]
        data.low = chartdata['BidLow']
        data.open = chartdata['BidOpen']
        data.close = chartdata['BidClose']
        data.high = chartdata['BidHigh']
        
        layout = self.chart.layout
        layout.xaxis.tickvals = [ii for ii in data.x if 
                                 chartdata.iloc[ii]['DateTime'].time() == datetime.time(9, 30)]
        xaxisdatetimes = [chartdata.iloc[ii]['DateTime'] for ii in layout.xaxis.tickvals]
        layout.xaxis.ticktext = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
                                 else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]
        
        layout.xaxis.hoverformat = ''
    
    def __init__(self, fullpricedata):
        self.fullpricedata = fullpricedata
        
        data = [ dict(
                type = 'candlestick')
               ]
        layout = dict(
                title = 'Candlestick Chart',
                xaxis = dict(
                    title = dict(
                        text = 'Time'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))
        self.chart = go.FigureWidget( data=data, layout=layout)
        
        # DEFINE WIDGETS AND CALLBACK FUNCTIONS
        self.startdatewidget = ipywidgets.DatePicker(
            description='Start Date',
            value=datetime.date(2019,1,2),
            disabled=False
        )
        self.enddatewidget = ipywidgets.DatePicker(
            description='End Date',
            value=datetime.date(2019,1,2),
            disabled=False
        )
        self.timeframewidget = ipywidgets.IntText(
            description='Timeframe (minutes)',
            value=1,
            disabled=False
        )
        self.widgets = [self.startdatewidget, self.enddatewidget, self.timeframewidget]
        for widget in self.widgets:
            widget.observe(self.updateChart, names = 'value')
        
        self.widgetsdisplay = ipywidgets.VBox([ipywidgets.HBox([self.startdatewidget, self.enddatewidget]),
                                        self.timeframewidget])
        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart, self.widgetsdisplay])
        
        self.updateChart(0)

In [6]:
candle = CandlestickChart(pricedata)
candle.display

    'data': [{'close': array([ 99.66,  99.78,  99.75, ..., 101.38, 101.54, 101.4…

In [7]:

class Backtester:
    def __init__(self, price_data, starting_cash=None, trading_strategy=None):
        if starting_cash == None:
            self.starting_cash = 10000
        else:
            self.starting_cash = starting_cash
        if trading_strategy == None:
            self.trading_strategy = self.basic_trading_strategy
        else:
            self.trading_strategy = trading_strategy
        self.price_data = price_data
    
    def backtest_trading_strategy(self, strategy_params):
        cash = {'High': self.starting_cash, 'Low': self.starting_cash, 'Open': self.starting_cash,\
                'Close': self.starting_cash, 'Best': self.starting_cash, 'Worst': self.starting_cash}
        shares = {'High': 0, 'Low': 0, 'Open': 0, 'Close': 0, 'Best': 0, 'Worst': 0}
        shares_value = {'High': 0, 'Low' :0, 'Open':0, 'Close': 0, 'Best': 0, 'Worst': 0}
        net_value = dict(cash)

        cash_list = list()
        shares_list = list()
        shares_value_list = list()
        net_value_list = list()

        commission = 0
        threshold = 0.01
        scale = 1

        for index, row in pricedata.iterrows():
            trading_dict = self.trading_strategy(strategy_params, index)
            buy = trading_dict['buy']
            sell = trading_dict['sell']
            quantity = trading_dict['quantity']

            for k in cash.keys():
                
                if k == 'Best':
                    bid_price = row['BidHigh']
                    ask_price = row['AskLow']
                elif k == 'Worst':
                    bid_price = row['BidLow']
                    ask_price = row['AskHigh']
                else:
                    bid_price = row['Bid' + str(k)]
                    ask_price = row['Ask' + str(k)]
                    
                if buy and cash[k] > commission:
                    cost = quantity*ask_price
                    cost = min(cost, cash[k])
                    quantity = cost/ask_price
                    cash[k] -= cost
                    shares[k] += quantity
                elif sell and shares[k] > 0:
                    quantity = min(quantity, shares[k])
                    cash[k] += quantity*bid_price
                    shares[k] -= quantity
                cash[k] -= commission

                # Careful with this rounding
                cash[k] = round(cash[k]*100)/100
                shares_value[k] = round(bid_price*shares[k])
                net_value[k] = round(cash[k] + shares_value[k])

            cash_list.append(dict(cash))
            shares_list.append(dict(shares))
            shares_value_list.append(dict(shares_value))
            net_value_list.append(dict(net_value))        
            #print('Cash: ' + str(cash['Open']) + ', Shares: ' + str(shares['Open']) + ', Net Value: ' + str(net_value['Open']))
        return {'cash': cash_list, 'shares': shares_list, 'shares_value': shares_value_list, 'net_value': net_value_list}
            
    # All trading strategies should take the current price data and strategy params as an input and output buy/sell and quantity
    def basic_trading_strategy(self, strategy_params, index):
        threshold = strategy_params['threshold']
        scale = strategy_params['scale']
        buy = 0
        sell = 0
        quantity = 0
        
        row = self.price_data.loc[index]
        if index > 0:
            last_row = self.price_data.loc[index-1]
        else:
            last_row = self.price_data.loc[0]

        delta = row['BidOpen'] - last_row['BidClose']
        
        if delta > threshold:
            buy = 1
            quantity = scale*round(100*delta)
        elif delta < -threshold:
            sell = 1
            quantity = scale*round(-100*delta)
        return {'buy': buy, 'sell': sell, 'quantity': quantity}

In [8]:
strategy_params = {'threshold': 0.01, 'scale': 5}
back_tester = Backtester(price_data=pricedata, starting_cash=10000, trading_strategy=None)
back_tester.trading_strategy = back_tester.basic_trading_strategy
portfolio_dict = back_tester.backtest_trading_strategy(strategy_params)

In [9]:
class ProfitCharts():
    def __init__(self, portfolio_dict, key):
        self.portfolio = portfolio_dict
        cash = [d[key] for d in portfolio_dict['cash']]
        shares = [d[key] for d in portfolio_dict['shares']]
        shares_value = [d[key] for d in portfolio_dict['shares_value']]
        net_value = [d[key] for d in portfolio_dict['net_value']]

        data1 = [ dict(
                type = 'scatter',
                x = [ii for ii in range(len(pricedata['BidLow']))],
                y = cash),

                dict(
                type = 'scatter',
                x = [ii for ii in range(len(pricedata['BidLow']))],
                y = shares_value),

                dict(
                type = 'scatter',
                x = [ii for ii in range(len(pricedata['BidLow']))],
                y = net_value)
               ]

        layout1 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'Time'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))

        self.chart1 = go.FigureWidget( data=data1, layout=layout1)
        
        data2 = [
                dict(
                type = 'scatter',
                x = [ii for ii in range(len(pricedata['BidLow']))],
                y = [nv - net_value[0] for nv in net_value])
               ]

        layout2 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'Time'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Price'
                    )
                ))

        self.chart2 = go.FigureWidget( data=data2, layout=layout2)
        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart1, self.chart2])

In [10]:
profit_charts = ProfitCharts(portfolio_dict=portfolio_dict, key='Open')
profit_charts.display

    'data': [{'type': 'scatter',
              'uid': 'ad0e69e6-5d5a-4ad4-a008-1…